# Pre-process Cardzone Dataset

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Data Collection and Investigation

__Import from SQL__ <br>
2 sample rule to indicate the fraud label
    1. High Risk Countries based on currency code
    2. Transaction that occured from midnight until 5am

__Currency Code__ <br>
971 - AFGHANISTAN <br>
643 - RUSSIAN <br>
586 - PAKISTAN

__SQL Statement__ <br>
UPDATE `cz_authtxn` <br>
SET `AUTHTXN_FRAUD_CHECK` = CASE <br>
&emsp; WHEN AUTHTXN_CURRENCY_CODE = 971 AND AUTHTXN_TRXN_TIME < 050000 THEN 'D' <br>
&emsp; WHEN AUTHTXN_CURRENCY_CODE = 643 AND AUTHTXN_TRXN_TIME < 050000 THEN 'D' <br>
&emsp; WHEN AUTHTXN_CURRENCY_CODE = 586 AND AUTHTXN_TRXN_TIME < 050000 THEN 'D' <br>
&emsp; ELSE 'F' <br>
END

In [ ]:
# Input data files are available in the "../input/" directory
df = pd.read_csv('D:/Python Project/Credit Card Fraud Detection/cardzone dataset/cz_authtxn.csv', skipinitialspace=True)
    
# print number of records in the dataset
print("Number of records:" , len(df))
print("Number of features:" , len(df.columns))

# check is there any null value in cells at columns
print("Number of features that has empty cells" , len(df.columns[df.isna().any()].tolist()))
df.columns[df.isna().any()].tolist()


In [ ]:
# Checks how many fraud in this dataset
print(df['AUTHTXN_FRAUD_CHECK'].value_counts())

print('\nFraud is {}% of our data.'.format(df['AUTHTXN_FRAUD_CHECK'].value_counts()['D'] / float(df['AUTHTXN_FRAUD_CHECK'].value_counts()['F'])*100))


In [ ]:
# Check Class variables that has 0 value for Genuine transactions and 1 for Fraud
fig, ax = plt.subplots(1, 1)
ax.pie(df.AUTHTXN_FRAUD_CHECK.value_counts(),autopct='%1.1f%%', labels=['Legitimate','Fraud'], colors=['yellowgreen','r'])
plt.axis('equal')
plt.ylabel('')

Conclusion = Imbalance. Learning of the data highly bias 

# Data Preprocessing
## Deal with missing values
There are several strategies to deal with missing data and there is no exact right answer
- A value from another randomly selected record.
- A mean, median or mode value for the column. (reduces variance in the dataset)
    - Categorical NaNs for mode
    - Numerical NaNs for mean
    - If there are outliers in Numerical, try median (less sensitive to them)
- Drop those record / column
- A value estimated by another predictive model.
- A distint constant value, such as 0 or -9999

However, missing values does not necessarily means to missing information. <br>
For example, if someone does not own a car, then of course it has no color resulting in missing value <br>
If replace the missing value with some other value might leads to wrong result

### Drop columns which has over 70% NaN values 
If a particular column has over 60% or 70% NaN values, it is better to drop it because it does not contributes towards giving information to ML model 
https://www.researchgate.net/publication/239608247_Machine_Learning_Based_Missing_Value_Imputation_Method_for_Clinical_Datasets

In [ ]:
# First, print out the exact 30% records number that i want to keep the column
print("Number of records that i want to keep in columns : ", len(df) * 0.3)

df.isna().mean().round(4) * 100

# Now, drop any columns that has 70% NaN
df.dropna(thresh=0.3*len(df), axis=1, inplace=True)


In [ ]:
# print list of columns after initial drop
print("The number of columns after initial dropping is ", len(df.columns))
print(df.columns)

# Recheck still got any left out blank cell in the columns that does not meet threshold
print("\nThe number of blank records in columns after initial dropping is ", len(df.columns[df.isna().any()]))
df.columns[df.isna().any()].tolist()


### Drop unnecessary columns according to prototype design document

In [ ]:
drop_columns = ['AUTHTXN_NO', 'AUTHTXN_CARDHOLDER_NAME', 'AUTHTXN_SYSTEM_ID', 'AUTHTXN_APPROVED_AMT', 'AUTHTXN_STAN',
                'AUTHTXN_PREV_STAN', 'AUTHTXN_TRANS_DATETIME', 'AUTHTXN_REQUEST_DATE', 'AUTHTXN_REQUEST_TIME', 'AUTHTXN_RESPONSE_DATE',
                'AUTHTXN_RESPONSE_TIME', 'AUTHTXN_SETTLED_DATE', 'AUTHTXN_LAST_UPDATE_DATE', 'AUTHTXN_LAST_UPDATE_TIME', 'AUTHTXN_CARD_EXPIRY_DATE',
                'AUTHTXN_POS_COND_CODE', 'AUTHTXN_RETRIEVAL_REFNO', 'AUTHTXN_OLD_RETRIEVAL_REFNO', 'AUTHTXN_APPROVAL_CODE', 'AUTHTXN_RESPONSE_CODE',
                'AUTHTXN_MERCHANT_NAME', 'AUTHTXN_SETTLED_IND', 'AUTHTXN_AUTO_EXPIRY_DATE', 'AUTHTXN_SUBSIDY_REBATE_AMT', 'AUTHTXN_MERC_MDR_AMT',
                'AUTHTXN_MERC_COMM_AMT', 'AUTHTXN_PROCESSEDBY', 'AUTHTXN_TYPE', 'AUTHTXN_INTERCHG_IND', 'AUTHTXN_GEOGRAPHY_IND',
                'AUTHTXN_BONUS','AUTHTXN_FEE', 'AUTHTXN_ACQ_CHARGE_AT_IND', 'AUTHTXN_POST_IND', 'AUTHTXN_COMPONENT_ID',
                'AUTHTXN_MTI', 'AUTHTXN_PROC_CD', 'AUTHTXN_BONUS_POINT', 'AUTHTXN_TERMBONUS_POINT', 'VERSION',
                'AUTHTXN_SUBSIDY_REBATE_QTY', 'AUTHTXN_STMT_INC_BONUS', 'AUTHTXN_STMT_INC_FEE', 'AUTHTXN_STMT_INC_COST', 'AUTHTXN_STMT_INC_COMM',
                'AUTHTXN_EBONUS', 'AUTHTXN_EFEE', 'AUTHTXN_SERVICE_CODE', 'AUTHTXN_ALT_RESPONSE_CODE', 'AUTHTXN_GST_AMT',
                'AUTHTXN_MERC_GST_AMT', 'AUTHTXN_VTXNTYPGRP_ID', 'AUTHTXN_EDC_SETTLED_IND', 'AUTHTXN_VS_TRXN_ID', 'AUTHTXN_MATCH_PREVTXN_IND',
                'AUTHTXN_ACQ_INST_ID', 'AUTHTXN_EDC_SETTLED_DATE', 'AUTHTXN_INTERBRANCH_IND', 'AUTHTXN_PIN_BASED', 'AUTHTXN_EXP_IND',
                'AUTHTXN_FOREX_MARKUP_AMT', 'AUTHTXN_EXCESS_AMT']

for col in drop_columns:
    df.drop(col, axis=1, inplace=True)

In [ ]:
print("The number of columns after second dropping is ", len(df.columns))
print(df.columns)

print("\nThe number of blank records in columns after second dropping is ", len(df.columns[df.isna().any()]))
df.columns[df.isna().any()].tolist()


In [ ]:
# print in a graphical way to visualize
import missingno as msno

msno.bar(df.sample(3000))


In [ ]:
# check unique values count in columns
df.nunique()

In [ ]:
# Drop columns if only have one unique value
for col in df.columns:
    if len(df[col].unique()) == 1:
        df.drop(col,inplace=True,axis=1)

In [ ]:
print("After dropping columns which has only 1 unique values")
print("========================================================")
print("Number of records:" , len(df))
print("Number of features:" , len(df.columns))

In [ ]:
# Replace 0 to the missing values in colum
columns_nan_value = df.loc[:, df.isna().sum() > 0].columns

for i, col in enumerate(columns_nan_value):
    df[col].fillna(value = 0, inplace=True)

In [ ]:
# Recheck for any blank columns
df.isna().sum() > 0


### Convert date and time column to readable DateTime object
First, convert their types into String, then preprocess them as follows:
- For Dates which dont have year, assume it as 2018
- For Time(24-hour) which dont have Hours and Minutes infront, append the time with 00 : 00 : xx <br> Because of Excel automatically delete initial 0s infront, 

In [ ]:
df.AUTHTXN_TRXN_DATE = df.AUTHTXN_TRXN_DATE.astype(str)
df.AUTHTXN_TRXN_TIME = df.AUTHTXN_TRXN_TIME.astype(str)


In [ ]:
for i, date in enumerate(df.AUTHTXN_TRXN_DATE):
    if len(date) == 3:
        new_date = pd.datetime.strptime(date, '%m%d').date()
        new_date = new_date.replace(2018, 1)
        df.at[i, 'AUTHTXN_TRXN_DATE'] = new_date
    elif len(date) == 4:
        new_date = pd.datetime.strptime(date, '%m%d').date()
        new_date = new_date.replace(2018)
        df.at[i, 'AUTHTXN_TRXN_DATE'] = new_date
    elif len(date) == 8:
        new_date = pd.datetime.strptime(date, '%Y%m%d').date()
        df.at[i, 'AUTHTXN_TRXN_DATE'] = new_date


In [ ]:
for i, time in enumerate(df.AUTHTXN_TRXN_TIME):
    if len(time) == 1 or len(time) == 2:
        new_time = pd.datetime.strptime(time, '%S').time()
        df.at[i, 'AUTHTXN_TRXN_TIME'] = new_time
    elif len(time) == 3:
        time = str(0) + time
        new_time = pd.datetime.strptime(time, '%M%S').time()
        df.at[i, 'AUTHTXN_TRXN_TIME'] = new_time
    elif len(time) == 4:
        new_time = pd.datetime.strptime(time, '%M%S').time()
        df.at[i, 'AUTHTXN_TRXN_TIME'] = new_time
    elif len(time) == 5:
        time = str(0) + time
        new_time = pd.datetime.strptime(time, '%H%M%S').time()
        df.at[i, 'AUTHTXN_TRXN_TIME'] = new_time
    elif len(time) == 6:
        new_time = pd.datetime.strptime(time, '%H%M%S').time()
        df.at[i, 'AUTHTXN_TRXN_TIME'] = new_time
        

In [ ]:
pd.to_datetime(df.AUTHTXN_TRXN_TIME, format='%H:%M:%S').dt.hour.value_counts().sort_index().plot() 

In [ ]:
df["Year"] = pd.to_datetime(df.AUTHTXN_TRXN_DATE).dt.year
df["Month"] = pd.to_datetime(df.AUTHTXN_TRXN_DATE).dt.month
df["Day"] = pd.to_datetime(df.AUTHTXN_TRXN_DATE).dt.day
df["Day_of_week"] = pd.to_datetime(df.AUTHTXN_TRXN_DATE).dt.dayofweek

df["Hour"] = pd.to_datetime(df.AUTHTXN_TRXN_TIME, format='%H:%M:%S').dt.hour

# Period of time labeling
hours = df['Hour']
bins = [-1, 4, 8, 16, 20]
labels = ['Midnight', 'Morning','Afternoon','Evening','Night']
df['Period_of_time']  = np.array(labels)[np.array(bins).searchsorted(hours)-1]

In [ ]:
df.drop(['AUTHTXN_TRXN_DATE'], axis=1, inplace=True)
df.drop(['AUTHTXN_TRXN_TIME'], axis=1, inplace=True)

### Move AUTHTXN_FRAUD_CHECK as labels to last index for easy see

In [ ]:
# first, map the D as 1, F as 0
df['AUTHTXN_FRAUD_CHECK'] = df['AUTHTXN_FRAUD_CHECK'].map({'F': 0, 'D': 1})


In [ ]:
fraud_label_df = df.AUTHTXN_FRAUD_CHECK
df.drop(['AUTHTXN_FRAUD_CHECK'], axis=1, inplace=True)
df['AUTHTXN_FRAUD_CHECK'] = fraud_label_df


In [ ]:
df.head()


In [ ]:
for col in df.columns:
    if col not in ['AUTHTXN_REQUEST_AMT', 'AUTHTXN_NET_AMT', 'AUTHTXN_BILLING_TXN_AMT']:
        if(df[col].dtype == np.float64 or df[col].dtype == np.int64):
            df[col] = df[col].astype(str)
        elif(df[col].dtype == np.float32 or df[col].dtype == np.int32):
            df[col] = df[col].astype(str)

### Export the pre-processed dataset and fraud set (AUTHTXN_FRAUD_CHECK as D only)

In [ ]:
df.to_csv(r'D:\Python Project\Credit Card Fraud Detection\cardzone dataset\preprocessed_dataset_3.csv', index=False)
# df.loc[df['AUTHTXN_FRAUD_CHECK'] == 'D'].to_excel(r'D:\Python Project\Credit Card Fraud Detection\cardzone dataset\fraud_set.xlsx',
#                                                 index=False)